# ML Pipeline Orchestration

There are several tools available for orchestrating an ML pipeline. [Airflow](https://airflow.apache.org/) is very popular, and is a powerful general purpose tool, and better than a simple scheduler, but may not be the best choice for ML pipelines. [Prefect](https://www.prefect.io/) is a popular alternative to look into as well. For this session, we will be looking at [Metaflow](https://metaflow.org/), an open source tool created at Netflix specifically to make it easier for non-engineering data scientists to create workflows.

Note that Metaflow also does experiment tracking, similar to MLFlow. Also note that Metaflow is in active development, and is therefore more likely to have bugs, or changes, but they do have an active [Slack channel](http://slack.outerbounds.co/).

The *problem* with orchestration engines is that they are complex, and we can easily spend many weeks learning how to use just one of these tools, and then there is no guarantee that the company you end up working at will be using the same tool. Because of this, we will only scratch the surface of Metaflow in order to understand and appreciate the idea of setting up *flows* for our work, without getting too bogged down in all of the details of scaling up and out. Let's keep it simple and save the struggle for when you're actually getting paid to wrestle with technology.



## Metaflow Basics

First, we can install Metaflow and check that it gets installed properly:

`pip install metaflow`  
`metaflow`  
`metaflow --help`   

We should add this to our requirements.txt file.

We can define a flow (DAG) as a class which will be derived from metaflow's `FlowSpec` class, and we can define a step (node) of that flow as a method of our class (which we can name whatever we want), which will be annotated with the `@step` decorator. Here's a quick example:

```
from metaflow import FlowSpec, step

class HelloWorldFlow(FlowSpec):
    @step
    def start(self):
        """Starting point"""
        print("This is start step")
        self.next(self.hello)

    @step
    def hello(self):
        """Say hi"""
        print("Hello World!")
        self.next(self.end)

    @step
    def end(self):
         """Finish line"""
         print("This is end step")

if __name__ == '__main__':
     HelloWorldFlow()
```

Let's pick this apart. We created our own flow, which we called `HelloWorldFlow`, as a subclass of `FlowSpec`. Our flow has three steps denoted by the `@step` decorator. We explicitly define a `start` and `end` step. Note that the first step must be called `start` and the last step must be called `end`. We define the edges of our DAG, how one step flows to the next, by calling `next(step_name)` where `step_name` is the next step/node in the flow/DAG. We can only have one flow in a single python file, and we instantiate the flow using `if __name__ == '__main__':` so that it runs if we call the file as a script. We can save all of this into a python file called `helloworld.py` and then run:

`python helloworld.py`

Metaflow will not run the flow, it will simply check that the DAG is valid, and will run a linter to check our code. We will need to `pip install pylint` for this to work. We should see that the DAG is good and that pylint is happy. Let's go back and make the DAG invalid and rerun the code to see what happens. 

We can see a representation of the DAG by running 

`python helloworld.py show`

After verifying the DAG is what we want, we can run the flow:

`python helloworld.py run`

This will run the flow and we will see lines of output like this:

`2023-03-09 13:01:19.142 [1678395679097777/start/1 (pid 86303)] Task is starting.`

These lines contain a

- timestamp: 2023-03-09 13:01:19.142  
- run ID: 1678395679097777  
- step name: start  
- task ID: 1  
- process ID: pid 86303  
- log message: Task is starting.

All of these IDs help us track our experiments. Each run of the flow gets a unique run ID, and each task is processed separately and gets their own process ID. The combination of a run ID, step name, and task ID defines a *pathspec* which uniquely identifies a task and might come in handy later.



### Data flow

When processing data we need to be cognizant of how our workflow will access input data, and how that data will flow through the workflow. Commonly, in data science workflows, we would like to persist a state between steps in the workflow, meaning that we don't want each step of the workflow to be independent of others steps, we want to transfer the state of each step to the next step in the workflow. 

Recall that in Jupyter we can run code cells out of order, which means that unexpected things may change. Defining a workflow helps to alleviate that issue, and ensures that code is run in a specific order, and that variables that were defined in a previous step will persist to subsequent steps. With Metaflow we can persist states by using a common data store, which is managed internally by Metaflow. 

Persisting data is as simple as creating *instance* variables, which means, all we have to do is assign any variables or data that we want to persist to `self`. Anything assigned to `self` is also called an *artifact*, and these artifacts can be any python object (scalar, model, dataframe, etc.).

It is important to persist states in this way because each task in the workflow is executed separately, which may happen on different physical CPUs. Here's an example of persisting state:

```
from metaflow import FlowSpec, step

class Counterflow(FlowSpec):
    @step
    def start(self):
        self.count = 0
        self.next(self.add)

    @step
    def add(self):
        print("The count is", self.count, "before incrementing")
        self.count += 1
        self.next(self.end)

    @step
    def end(self):
        self.count += 1
        print("Final count is", self.count)

if __name__ == '__main__':
    Counterflow()
```

After running the above we can check logs or variables for a specific task using either the `logs` or `dump` command.

`python counterflow.py dump 1678397687773439/end/3`  
`python counterflow.py logs 1678397687773439/end/3`

What we should see, if we run `dump` or `logs` on the `end` task is that count = 2.



### Parameters

We can create parameters for our flows so that we can run the same flow with different values of those parameters. For example, if we define a model training flow for a random forest, we may want to create a parameter for the number of trees, or the number of features we sample for each node split. Then, when we run the flow, we can pass the parameter values that we want to use. 

Defining parameters in a flow is as simple as:

```
from metaflow import FlowSpec, step, Parameter

class Counterflow(FlowSpec):

    begin_count = Parameter('ct', default = 20, type = int, required = True)

    @step
    def start(self):
        self.count = self.begin_count
        self.next(self.add)

    ...

```

All arguments to `Parameter` are optional arguments, and default to `None`. 

- `help`: a line of help text  
- `required`: is this parameter required?  
- `default`: what is the default value?  
- `type`: what is the type? Can be `str, float, int, bool`.

If we saved the above (with the rest of the code from counterflow.py), we can run the flow, with a parameter value, like this:

`python counterflow.py run --ct 50`

Notice, the parameter name is `ct`, and we assigned it to a variable called `begin_count` in the script. We could've simply named these the same thing if we wanted to keep things simple.



### Branching and Merging 

For DAGs that are more complex than linear, such as nodes that flow to, or flow from, more than one node, we can define these in Metaflow easily.

In the case of branching, where a node flows into more than one node, we can simply include all of the nodes in the `next` method:

`self.next(self.one, self.two)` means that after the current step is run, steps `one` and `two` will run next, independently. Note that if there is any branching in our flow, we must do a merge at some point, before we reach the end of the flow. To do so, we can define a new step called `join` that will define how to merge the steps, like so:

`@step`
`def join(self, inputs):`

Note that we have an extra argument, called `inputs`, in a join step. Let's look at an example.

```
from metaflow import FlowSpec, step

class CounterBranchFlow(FlowSpec):
    @step
    def start(self):
        self.creature = "dog"
        self.count = 0
        self.next(self.add_one, self.add_two) # branching happens here

    @step
    def add_one(self):
        self.count += 1
        self.next(self.join) # move to the merging step, where the add_one and add_two nodes will merge

    @step
    def add_two(self):
        self.count += 2
        self.next(self.join) # move to the merging step, where the add_one and add_two nodes will merge

    @step 
    def join(self, inputs):
        self.count = max(inp.count for inp in inputs) # iterate over inputs to find max count
        print("count from add_one", inputs.add_one.count) # print specific values from named branch
        print("count from add_two", inputs.add_two.count) # print specific values from named branch

        self.creature = inputs[0].creature # creature was not modified, so we can use the index 0, and this step is required --- we are reassigning the self.creature variable. but if you had 10+ variables you could just : self.merge_artifacts(inputs)

        self.next(self.end)

    @step
    def end(self):
        print("The creature is", self.creature)
        print("The final count is", self.count)

if __name__ == '__main__':
    CounterBranchFlow()
```

When we run the above, we should notice that the process ID is different for `add_one` and `add_two`. This is because those two steps are executed separately, which will likely happen in parallel.

Merging can be complicated. What if we have a lot more artifacts to deal with? It is true that we **must** reassign **all** artifacts in the join step. Luckily, we can use `self.merge_artifacts(inputs)` to reassign all artifacts that we don't explicitly reassign. 

### foreach

Suppose instead of having static branches (a fully known DAG) we have dynamic branches, which means we have a DAG where we need to *fan out* a node into many nodes to do a single task in parallel, for example training a random forest model on different *folds* of the training data. To create these types of dynamic branches in Metaflow, we can do so by using the `foreach` argument in `next`.

```
from metaflow import FlowSpec, step

class ForeachFlow(FlowSpec):

    @step
    def start(self):
        self.creatures = ['bird', 'mouse', 'dog']
        self.next(self.analyze_creatures, foreach='creatures') # referring to our list of creatures, create a node (task) for each creature

    @step
    def analyze_creatures(self):
        print("Analyzing", self.input) # as soon as you do foreach (in the previous step), metaflow automatically assigns each variable as self.input as it comes in 
        self.creature = self.input
        self.score = len(self.creature)
        self.next(self.join)
    # this is what analyze_creatures creates as the "inputs": 
    
    #inputs = [
    <StepOutput: self.creature = "bird", self.score = 4>,
    <StepOutput: self.creature = "mouse", self.score = 5>,
    <StepOutput: self.creature = "dog", self.score = 3>]

    @step 
    def join(self, inputs):
        self.best = max(inputs, key=lambda x: x.score).creature # similar to argmax, this will return which input has maximum score, syntax is a bit odd 
        self.next(self.end)

    @step
    def end(self):
        print(self.best, 'won!')

if __name__ == '__main__':
    ForeachFlow()
```

It is possible that we might get carried away, and accidentally run many, many processes in parallel. We can control the concurrency by setting the `max-num-splits` parameter. For example: 

`python foreach.py run --max-num-splits 100` #imagine if we were trying to parallelize 10000 tasks this would put cap it at 100

Or we can set an environmental variable: `export METAFLOW_RUN_MAX_NUM_SPLITS=100`.



## Example

Let's use our favorite wine dataset to demonstrate how we can use Metaflow for training a model. 

```
from metaflow import FlowSpec, step

class ClassifierTrainFlow(FlowSpec):

    @step
    def start(self):
        from sklearn import datasets
        from sklearn.model_selection import train_test_split

        X, y = datasets.load_wine(return_X_y=True)
        self.train_data, self.test_data, self.train_labels, self.test_labels = train_test_split(X,y, test_size=0.2, random_state=0)
        print("Data loaded successfully")
        self.next(self.end)

    @step
    def end(self):
        self.model = 'nothing_yet'
        print('done')

if __name__=='__main__':
    ClassifierTrainFlow()
```

We can save the above as `classifier_train_v1.py` and run it, and then confirm that data was loaded using the `dump` command:

`python classifier_train_v1.py dump 1678478123816769/start/1`

We should see that yes, some data is there, but we cannot see it because it is too large. In a notebook, we can inspect artifacts more easily. If we copy and paste the following into a notebook, we will be able to actually see the data.

```
from metaflow import Flow
run = Flow('ClassifierTrainFlow').latest_run
run['start'].task.data.train_data
```

From here, let's write a scoring script for getting predictions from our model (which has not yet been trained, but that's ok).

```
from metaflow import FlowSpec, step, Flow, Parameter, JSONType

class ClassifierPredictFlow(FlowSpec):
    vector = Parameter('vector', type=JSONType, required=True)

    @step
    def start(self):
        run = Flow('ClassifierTrainFlow').latest_run 
        self.train_run_id = run.pathspec 
        self.model = run['end'].task.data.model
        print("Input vector", self.vector)
        self.next(self.end)

    @step
    def end(self):
        print('Model', self.model)

if __name__=='__main__':
    ClassifierPredictFlow()
```

If we run the above flow and pass a vector to it, what should happen?

`python classifier_predict_v1.py run --vector '[1,2]'`

Now that we can verify that our two scripts are connected, we can worry about training the actual model, using the code below. Suppose we want to try a couple of different algorithms, kNN and SVM, we can do branching to train both in parallel, and then we can merge the results in a new step called `choose_model` where we pick the model with the best fit. We can also now integrate MLFlow to register our final model.

```
from metaflow import FlowSpec, step

class ClassifierTrainFlow(FlowSpec):

    @step
    def start(self):
        from sklearn import datasets
        from sklearn.model_selection import train_test_split

        X, y = datasets.load_wine(return_X_y=True)
        self.train_data, self.test_data, self.train_labels, self.test_labels = train_test_split(X,y, test_size=0.2, random_state=0)
        print("Data loaded successfully")
        self.next(self.train_knn, self.train_svm)

    @step
    def train_knn(self):
        from sklearn.neighbors import KNeighborsClassifier

        self.model = KNeighborsClassifier()
        self.model.fit(self.train_data, self.train_labels)
        self.next(self.choose_model)

    @step
    def train_svm(self):
        from sklearn import svm

        self.model = svm.SVC(kernel='poly')
        self.model.fit(self.train_data, self.train_labels)
        self.next(self.choose_model)

    @step
    def choose_model(self, inputs):
        import mlflow
        mlflow.set_tracking_uri('https://mlflow-test-run-275570243848.us-west2.run.app')
        mlflow.set_experiment('metaflow-experiment')

        def score(inp):
            return inp.model, inp.model.score(inp.test_data, inp.test_labels)

        self.results = sorted(map(score, inputs), key=lambda x: -x[1])
        self.model = self.results[0][0]
        with mlflow.start_run():
            mlflow.sklearn.log_model(self.model, artifact_path = 'metaflow_train', registered_model_name="metaflow-wine-model")
            mlflow.end_run()
        self.next(self.end)

    @step
    def end(self):
        print('Scores:')
        print('\n'.join('%s %f' % res for res in self.results))
        print('Model:', self.model)


if __name__=='__main__':
    ClassifierTrainFlow()
```

We can save the above as `classifier_train.py` and run it to see that it works. After confirming that it runs without error, we can modify our scoring script.

```
from metaflow import FlowSpec, step, Flow, Parameter, JSONType

class ClassifierPredictFlow(FlowSpec):
    vector = Parameter('vector', type=JSONType, required=True)

    @step
    def start(self):
        run = Flow('ClassifierTrainFlow').latest_run 
        self.train_run_id = run.pathspec 
        self.model = run['end'].task.data.model
        print("Input vector", self.vector)
        self.next(self.end)

    @step
    def end(self):
        print('Model', self.model)
        print('Predicted class', self.model.predict([self.vector])[0])

if __name__=='__main__':
    ClassifierPredictFlow()
```

Let's save the above as `classifier_predict.py`. To run it, we need to pass a vector of values so we can get a prediction from the model that we chose during our last run.

`python classifier_predict.py run --vector '[14.3,1.92,2.72,20.0,120.0,2.8,3.14,0.33,1.97,6.2,1.07,2.65,1280.0]'`

To fully examine the artifacts (the trained model), we can go into a notebook like before, and extract the fitted model from the last run.

```
from metaflow import Flow
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

run = Flow('ClassifierTrainFlow').latest_run
run

model, score = run.data.results[0]
test_data = run['start'].task.data.test_data
test_labels = run['start'].task.data.test_labels

predictions = model.predict(test_data)
cm = confusion_matrix(test_labels, predictions, labels=model.classes_)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()
```

Let's take a second to appreciate what we just accomplished. We created a training pipeline that loads data, trains two models, and selects the best model. We then created a scoring pipeline that takes the best model from the training pipeline and scores a brand new unseen observation. Not too shabby!


In [3]:
from metaflow import FlowSpec, step

class ClassifierTrainFlow(FlowSpec):

    @step
    def start(self):
        from sklearn import datasets
        from sklearn.model_selection import train_test_split

        X, y = datasets.load_wine(return_X_y=True)
        self.train_data, self.test_data, self.train_labels, self.test_labels = train_test_split(X,y, test_size=0.2, random_state=0)
        print("Data loaded successfully")
        self.next(self.end)

    @step
    def end(self):
        self.model = 'nothing_yet'
        print('done')

# if __name__=='__main__':
#     ClassifierTrainFlow()

In [ ]:
# from metaflow import Flow
# run = Flow('ClassifierTrainFlow').latest_run
# run['start'].task.data.train_data

# doesn't work :(


### What Next?

Our toy dataset example is a nice way to demonstrate flows, but it leaves a lot of questions unanswered, such as:

- What if our input data is very big?  
- What if we want to train 1000s of models?  
- Can we modularize our code?  
- How do we schedule flows to run instead of running them manually?  
- How can we test out a newer version of sklearn if our current in-production model uses an older version?

We will try to answer these questions a future lab.

# ML Workflows Lab

## Overview

Creating ML flows is necessary when moving models out of development notebooks and into production-ready code. There are many ways to do this, but creating DAGs, whether through a tool, or through a collection of scripts and shell scripts that run in a specific order, is the default. 

## Goal

The goal in this lab is to create two flows, one for training our model (using the same dataset we selected for the previous MLFlow labs), and one for model scoring (sometimes called inference), and run these flows locally. We will likely modify our training flow later when we move Metaflow to GCP, and our scoring flow later when we get to the model deployment section of the course. We will also integrate MLFlow model registration with our flow.

## Instructions

### Installation and Creating a Training Flow

1. Install Metaflow in your project environment and add it to the requirements.txt file.  
2. Use the `FlowSpec` class and the `@step` decorator to create a flow that will train your model on your local machine. You *might* have steps such as: ingest raw data; feature transformations; hyperparameter tuning; choose best model; and register and save model using the MLFlow model registry (*with MLFlow running locally for now*). You can modularize your code so that different steps can be in different flows, or more commonly so that different steps of the same flow can be imported from other modules. For example, data transformations can be written in a separate script and imported into a specific step in your flow like this:  

```
@step
def start(self):
    import dataprocessing
```  
    
3. Be sure to add parameters to your flow when it makes sense, for example, for cross-validation or setting seeds.  
4. Save your flow in the `src` folder as `trainingflow.py`. Run your training flow to register a new "best" model in MLFlow using the parameters from your previously registered "best" model.  

### Creating a Scoring FLow

5. Create a model scoring flow for making predictions from your registered model on new data. You *might* have steps such as: ingest new data; feature transformations; load the registered model; make predictions; and output predictions. Again, you may have separate scripts for each of these steps which can be imported into the steps of the flow. 
6. Save your flow in the `src` folder as `scoringflow.py`. Run your scoring flow on a set of data that was not used for model training, or simply save a hold-out set of data from the training set and use this to demonstrate your scoring flow.  

### Turning It In

Push your flows to Github. **Do not add the `.metaflow/` folder to git**. Submit the Github URL that goes directly to the `src` folder. Be sure to add comments to your scripts so that they are easy to read.

In [22]:
import os
import pandas as pd
os.chdir("/Users/shruti/USF-Spring-2/ML-Ops/src/bank-loan")

In [37]:
# Load training data
train = pd.read_csv("credit_train.csv")
train = train.dropna(how='all') # Remove rows that have ONLY null values
train = train.drop(columns=["Loan ID", "Customer ID"]) # drop IDs

In [38]:
test = pd.read_csv("credit_test.csv")
test = test.dropna(how='all')
test = test.drop(columns=["Loan ID", "Customer ID"])

In [39]:
train.head()

,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,Fully Paid,445412.0,Short Term,709.0,1167493.0,8 years,Home Mortgage,Home Improvements,5214.74,17.2,NaN,6.0,1.0,228190.0,416746.0,1.0,0.0
1,Fully Paid,262328.0,Short Term,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,33295.98,21.1,8.0,35.0,0.0,229976.0,850784.0,0.0,0.0
2,Fully Paid,99999999.0,Short Term,741.0,2231892.0,8 years,Own Home,Debt Consolidation,29200.53,14.9,29.0,18.0,1.0,297996.0,750090.0,0.0,0.0
3,Fully Paid,347666.0,Long Term,721.0,806949.0,3 years,Own Home,Debt Consolidation,8741.90,12.0,NaN,9.0,0.0,256329.0,386958.0,0.0,0.0
4,Fully Paid,176220.0,Short Term,NaN,NaN,5 years,Rent,Debt Consolidation,20639.70,6.1,NaN,15.0,0.0,253460.0,427174.0,0.0,0.0


In [41]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


label_cols = ['Home Ownership', 'Purpose','Years in current job','Term']
encoders = {}

# Fit on training, transform both train and test
for col in label_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])  # Make sure no unseen categories
    encoders[col] = le

In [42]:
# Encode target separately
target_le = LabelEncoder()
train['Loan Status'] = target_le.fit_transform(train['Loan Status'])


In [43]:
from sklearn.model_selection import train_test_split
X = train.drop(columns = "Loan Status")
y = train["Loan Status"]

x_train, x_val, y_train, y_val = train_test_split(X,y, test_size=0.8, random_state= 25)

In [44]:
X_without_nulls = X.dropna()
y = y.loc[X_without_nulls.index]  # Make sure labels match new index

In [45]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_val_scaled = scaler.transform(x_val)

X_scaled = scaler.fit_transform(X_without_nulls)

In [84]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. Create the model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# 2. Fit the model on training data
rf.fit(x_train, y_train)

# 3. Make predictions on validation data
y_pred = rf.predict(x_val)

# 4. Evaluate the model
print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))


Accuracy: 0.8159333333333333

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.24      0.37     20333
           1       0.82      0.98      0.89     69667

    accuracy                           0.82     90000
   macro avg       0.82      0.61      0.63     90000
weighted avg       0.82      0.82      0.77     90000



In [74]:
test['Prediction'] = rf.predict(test)

# If you want to map predictions back to original Loan IDs
test_predictions = test['Prediction']
print(test_predictions.head())

0    1
1    1
2    1
3    1
4    1
Name: Prediction, dtype: int64


In [64]:
decoded_preds = target_le.inverse_transform(test_predictions)

In [85]:
rf.predict(test)

array([1, 1, 1, ..., 1, 1, 1])

In [89]:
decoded_preds = pd.Series(target_le.inverse_transform(rf.predict(test)))

In [66]:
pd.Series(decoded_preds).value_counts()

Fully Paid     9400
Charged Off     600
Name: count, dtype: int64

In [90]:
decoded_preds.value_counts()

Fully Paid     9338
Charged Off     662
Name: count, dtype: int64

In [21]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import numpy as np

In [31]:
mlflow.set_tracking_uri('sqlite:////Users/shruti/USF-Spring-2/ML-Ops/src/mlflow.db')
mlflow.set_experiment('bank-loan-experiment')

2025/04/20 14:45:56 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/04/20 14:45:56 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/Users/shruti/USF-Spring-2/ML-Ops/src/bank-loan/mlruns/1', creation_time=1745185556931, experiment_id='1', last_update_time=1745185556931, lifecycle_stage='active', name='bank-loan-experiment', tags={}>

<details><summary><b>Explanation: set_tracking_uri</b></summary>

- `sqlite:///mlflow.db` means the SQLite database file `mlflow.db` will be created in your **current working directory**.
- `sqlite:////full/path/to/your/directory/mlflow.db` (with **four** slashes) means the SQLite database file `mlflow.db` will be created at the **absolute path** `/full/path/to/your/directory/`.

The **fourth slash** is part of the standard URI convention when you're specifying an absolute file path in the system.
</details>

In [32]:
# Variable to track the best model and best accuracy
best_accuracy = -float('inf')
best_model = None
best_run_id = None

In [49]:
def objective(params):
    global best_accuracy, best_model, best_run_id
    with mlflow.start_run():
        classifier_type = params['type']
        del params['type']  # deleting type from parameters since when we unwrap it **params scikit learn doesn't accept a random "type" input

        if classifier_type == 'Decision Tree':
            clf = DecisionTreeClassifier(**params)
        elif classifier_type == 'Random Forest':
            clf = RandomForestClassifier(**params)
        elif classifier_type == 'Logistic Regression':
            clf = LogisticRegression(**params)
        else:
            return 0  
        
        # Automatic cross validation 
        acc = cross_val_score(clf, X_scaled, y, cv=5,scoring='accuracy').mean()

        if acc > best_accuracy:
            best_accuracy = acc
            best_model = clf 
            best_run_id = mlflow.active_run().info.run_id

        mlflow.set_tag("Model", classifier_type)
        mlflow.log_params(params)
        mlflow.log_metric("Accuracy", acc)
        
    mlflow.end_run()
    return {'loss': -acc, 'status': STATUS_OK}


search_space = hp.choice('classifier_type', [
    {
        'type': 'Decision Tree',
        'criterion': hp.choice('dtree_criterion', ['gini', 'entropy']),
        'max_depth': hp.choice('dtree_max_depth', [None] + list(range(1, 21))), 
        'min_samples_split': hp.randint('dtree_min_samples_split', 2, 20)
    },
    {
        'type': 'Random Forest',
        'n_estimators': hp.randint('rf_n_estimators', 20, 500),
        'max_features': hp.randint('rf_max_features', 2, 9),
        'criterion': hp.choice('criterion', ['gini', 'entropy'])
    },
    {
        'type': 'Logistic Regression',
        'C': hp.loguniform('lr_C', np.log(1e-5), np.log(1e5)),  
        'penalty': hp.choice('lr_penalty', ['l1', 'l2']),
        'solver': hp.choice('lr_solver', ['liblinear', 'saga']),
        'max_iter': hp.choice('lr_max_iter', [100, 200, 500, 1000, 2000])
    }
])

In [50]:
algo = tpe.suggest
trials = Trials()
best_result = fmin(
        fn=objective, 
        space=search_space,
        algo=algo,
        max_evals=32,
        trials=trials)

100%|██████████| 32/32 [29:10<00:00, 54.70s/trial, best loss: -0.8478032096816628] 


In [65]:
# After the search is complete, register the best model
if best_model is not None:
    mlflow.start_run(run_id=best_run_id)
    mlflow.set_tag("Model", "Best model")
    mlflow.log_metric("Accuracy", best_accuracy)
    # Register the model (you can use mlflow.sklearn or mlflow.pyfunc depending on your model type)

    mlflow.sklearn.log_model(best_model, "best_model")

    model_uri = f"runs:/{best_run_id}/best_model"
    mlflow.register_model(model_uri, "BankLoanModel")
    print(f"Best model registered with accuracy: {best_accuracy}")
    mlflow.end_run()

2025/04/20 18:57:29 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Best model registered with accuracy: 0.8478032096816628


Successfully registered model 'BankLoanModel'.
Created version '1' of model 'BankLoanModel'.


In [53]:
best_model

RandomForestClassifier(max_features=8, n_estimators=486)

In [57]:
best_run_id

'3538ce4094d9466c8f20b4c26b745a42'

In [58]:
model_uri

'runs:/3538ce4094d9466c8f20b4c26b745a42/best_model'

In [59]:
registed_model_name = "BankLoanModel"
version = 1  # Set to the version you want to load, or fetch the latest version dynamically

# Load the model from the model registry
loaded_model = f"models:/{registed_model_name}/{version}"
loaded_model

'models:/BankLoanModel/1'

In [70]:
bank_model = mlflow.sklearn.load_model(loaded_model)
bank_model

RandomForestClassifier(max_features=8, n_estimators=486)

In [66]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
client.get_registered_model("BankLoanModel")


<RegisteredModel: aliases={}, creation_timestamp=1745200649035, description=None, last_updated_timestamp=1745200649039, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1745200649039, current_stage='None', description=None, last_updated_timestamp=1745200649039, name='BankLoanModel', run_id='3538ce4094d9466c8f20b4c26b745a42', run_link=None, source='/Users/shruti/USF-Spring-2/ML-Ops/src/bank-loan/mlruns/1/3538ce4094d9466c8f20b4c26b745a42/artifacts/best_model', status='READY', status_message=None, tags={}, user_id=None, version=1>], name='BankLoanModel', tags={}>

In [67]:
client.get_latest_versions("BankLoanModel")


[<ModelVersion: aliases=[], creation_timestamp=1745200649039, current_stage='None', description=None, last_updated_timestamp=1745200649039, name='BankLoanModel', run_id='3538ce4094d9466c8f20b4c26b745a42', run_link=None, source='/Users/shruti/USF-Spring-2/ML-Ops/src/bank-loan/mlruns/1/3538ce4094d9466c8f20b4c26b745a42/artifacts/best_model', status='READY', status_message=None, tags={}, user_id=None, version=1>]

In [69]:
client.list_registered_models()

AttributeError: 'MlflowClient' object has no attribute 'list_registered_models'